In [23]:
import pandas as pd
import os
import gpt_2_simple as gpt2
import numpy as np

In [24]:
files = os.listdir('./data/')

In [25]:
li = []

for filename in files:
    filename = './data/%s' % (filename)
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)

In [26]:
data = data.drop_duplicates()

In [27]:
data

,User,Text,Date,Favorites,Retweets,Mentions,HashTags
0,naosalvo,@_marcelocrvg grava hj mas não falaram ainda q...,2015-04-30 21:26:52+00:00,0,2,NaN,NaN
1,naosalvo,"Revelações dos bastidores do SBT, a vaga do @R...",2015-04-30 21:24:49+00:00,21,68,@roxmo,NaN
2,naosalvo,"Posso tentar, acho justo",2015-04-30 21:08:30+00:00,11,62,NaN,NaN
3,naosalvo,Ouuuu fui chamado pro teste de DNA no Ratinho ...,2015-04-30 21:04:26+00:00,4,70,NaN,NaN
4,naosalvo,Vou gravar o The Noite (não sei que dia vai pr...,2015-04-30 21:03:35+00:00,10,81,NaN,NaN
...,...,...,...,...,...,...,...
62967,jairbolsonaro,Indico ao eleitores de SP p/ que votem para de...,2014-10-04 10:38:11+00:00,13,28,NaN,NaN
62968,jairbolsonaro,Meu voto para deputado federal no RJ:,2014-10-04 10:28:01+00:00,27,43,NaN,NaN
62969,jairbolsonaro,Meus votos para deputado estadual e federal no...,2014-10-04 10:27:23+00:00,24,31,NaN,NaN
62970,jairbolsonaro,DEBATE MEDÍOCRE ATÉ PARA UMA DISPUTA DE PREFEI...,2014-10-03 20:23:10+00:00,51,66,NaN,NaN


In [28]:
print('#Tweets de cidcidoso: %d' % sum(data['User'] == 'naosalvo'))
print('#Tweets de bozo: %d' % sum(data['User'] != 'naosalvo'))

n_sample = min(sum(data['User'] == 'naosalvo'), sum(data['User'] != 'naosalvo'))

#Tweets de cidcidoso: 57839
#Tweets de bozo: 5133


In [29]:
cid = data[data['User'] == 'naosalvo'].sample(n_sample)
bozo = data[data['User'] != 'naosalvo'].sample(n_sample)

data_balanced = pd.concat([cid, bozo]).sample(frac=1)

In [30]:
data_balanced = pd.DataFrame(data_balanced['Text'])

In [31]:
data_balanced_modified = data_balanced

In [32]:
for i in range(data_balanced_modified.shape[0]):
    element = '%s%s%s' % ('<|startoftext|>', data_balanced_modified.iloc[i][0], '<|endoftext|>')
    data_balanced_modified.iloc[i][0] = element

In [33]:
data_balanced_modified.to_csv('model_data.csv', index=False)

In [34]:
txt_list = []

In [35]:
for index, row in data_balanced.iterrows():
    element = '%s%s%s' % ('<|startoftext|>', row[0], '<|endoftext|>')
    txt_list.append(element)

In [36]:
with open('data_model.txt', 'w') as file:
    for element in txt_list:
        file.write(element)
    file.close()

# Model

In [20]:
file_name = 'model_data.csv'

In [21]:
sess = gpt2.start_tf_sess()

In [22]:
gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=100,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=100,
              save_every=100
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 41.34it/s]

Loading dataset...


dataset has 666437 tokens
Training...
[10 | 228.65] loss=2.76 avg=2.76
interrupted
Saving checkpoint/run1/model-14


In [ ]:
gpt2.generate(sess,
              length=200,
              temperature=0.7,
              prefix='<|startoftext|>',
              truncate='<|endoftext|>',
              include_prefix=False
              )

In [5]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices()[3])

IndexError: list index out of range